In [8]:
import pandas as pd

In [26]:
data_path = 'data/train.csv'
# Nombres de las columnas que deseas cargar (4, 5, 10 a 16)

# Leemos solo las columnas deseadas
data = pd.read_csv(data_path, usecols=[5,6,11,12,13,14,15,16])
data.head()


,Type of Travel,Class,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service
0,Personal Travel,Eco Plus,1,5,3,5,5,4
1,Business travel,Business,3,1,3,1,1,1
2,Business travel,Business,2,5,5,5,5,4
3,Business travel,Business,5,2,2,2,2,2
4,Business travel,Business,3,4,5,5,3,3


In [29]:
numericas = []
categoricas = []
nums = []

for i, col in enumerate(data.columns):
    tipo = "categórico" if data.dtypes[col] == object else "numérico"
    valores_perdidos = data[col].isnull().sum()  # Contar valores NaN
        
    if valores_perdidos > 0:
        print(f'La columna "{col}" tiene {valores_perdidos} valores perdidos.')
        
    # Añadimos la columna a la lista correspondiente
    if tipo == "numérico":
        numericas.append(col)
        
    else:
        categoricas.append(col)
        nums.append(i)

# Imprimimos las listas al final
print("Columnas categóricas:", categoricas)
print("\nColumnas numéricas:", numericas)
print(nums)


Columnas categóricas: ['Type of Travel', 'Class']

Columnas numéricas: ['Gate location', 'Food and drink', 'Online boarding', 'Seat comfort', 'Inflight entertainment', 'On-board service']
[0, 1]


Columna: "Departure/Arrival time convenient" - Tipo: numérica
Columna: "Ease of Online booking" - Tipo: numérica
Columna: "Gate location" - Tipo: numérica
Columna: "Food and drink" - Tipo: numérica
Columna: "Online boarding" - Tipo: numérica
Columna: "Seat comfort" - Tipo: numérica
Columna: "Inflight entertainment" - Tipo: numérica
Columna: "On-board service" - Tipo: numérica
